In [1]:
# --- Environment setup and imports ---

import os, sys
from pathlib import Path

root = Path("../..").resolve()   
sys.path.append(str(root))

# sanity check
print("Project root:", root)

# --- Core libraries ---
import copy
import random
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset

# --- Default parameters (used in old loop or new config overrides) ---
number_of_clients = 3
number_of_rounds = 5

Project root: /Users/christophbout/Library/Mobile Documents/com~apple~CloudDocs/FH/5. Semester/Capstone Innovation Project/Code/Sec_Federated_Learning


In [2]:
from src.utils.config_utils import load_config, normalize_config
from src.utils.exp_utils import compute_exp_id, ensure_exp_dirs
from src.utils.logging_utils import JsonlLogger, get_git_commit
from src.orchestrator.pipeline import run_training_rounds

In [3]:
# --- Configure experiment (edit as needed) ---
overrides = [
    "model.name=smallcnn",
    "data.partition=iid",
    "data.num_clients=5",
    "data.clients_per_round=3",
    "training.rounds=2",
    "training.local_epochs=1",
    "training.batch_size=64",
    "training.lr=0.01",
    "encryption.scheme=dp_selective",
    "encryption.layers=L1",
    "encryption.noise_multiplier=0.5",
    "encryption.clip_norm=1.0",
]

cfg = load_config("../../configs/config.yaml", overrides)
cfg["commit"] = get_git_commit()
cfg

{'defaults': [],
 'project': {'name': 'dp_fedlearn', 'output_dir': 'reports'},
 'seed': 1,
 'commit': '42ff3a20d7fe354ef509b5a3aacd9f1c89ffd235',
 'data': {'dataset': 'CIFAR10',
  'num_clients': 5,
  'clients_per_round': 3,
  'partition': 'iid',
  'dirichlet_alpha': 0.5},
 'model': {'name': 'smallcnn', 'num_classes': 10},
 'training': {'rounds': 2,
  'local_epochs': 1,
  'batch_size': 64,
  'lr': 0.01,
  'optimizer': 'sgd'},
 'encryption': {'scheme': 'dp_selective',
  'layers': 'L1',
  'noise_multiplier': 0.5,
  'clip_norm': 1.0,
  'secure_aggregation': False},
 'attack': {'name': 'yin', 'recon_per_client': 8, 'max_iters': 1000},
 'logging': {'to_jsonl': True,
  'to_mlflow': False,
  'mlflow_tracking_uri': None,
  'level': 'INFO'},
 'orchestration': {'exp_id': None}}

In [4]:
exp_id = compute_exp_id(normalize_config(cfg))
exp_root = ensure_exp_dirs(cfg["project"]["output_dir"], exp_id)

logger = JsonlLogger(
    exp_root / "train.jsonl",
    extra={"exp_id": exp_id, "commit": cfg["commit"], "seed": cfg["seed"]},
)

print(f"Experiment ID: {exp_id}")
print(f"Output folder: {exp_root}")

Experiment ID: ef2b17ddf180bea3
Output folder: reports/ef2b17ddf180bea3


In [ ]:
results = run_training_rounds(cfg, exp_root, logger)
print("Results:", results)

100%|██████████████████████████████████████████████████████████████████| 170M/170M [00:15<00:00, 10.9MB/s]


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

train_log = pd.read_json(exp_root / "train.jsonl", lines=True)
display(train_log)

plt.plot(train_log["round"], train_log["top1"], marker="o")
plt.title("Top-1 Accuracy per Round")
plt.xlabel("Round")
plt.ylabel("Accuracy")
plt.grid(True)
plt.show()